In [53]:
from pymongo import MongoClient
import pandas as pd
import threading
import os
import requests
import BeautifulSoup
from BeautifulSoup import BeautifulSoup
db=MongoClient('10.247.69.18').crowd
global n0
def make_df(sort,n):
    cursor = db['gofundme_donated'].find({},{"url":1,'created_at':1,"_id":0},limit =n).sort("_id", sort)
    df =  pd.DataFrame(list(cursor))
    df.columns=['created_at','url']
    return df
def collect_donations(page):
    headers = {'content-type': 'application/json'}
    url='https://www.gofundme.com/mvc.php?route=donate/pagingdonationsb&url='+page+'&idx=1'
    response=requests.post(url, headers=headers)
    soup = BeautifulSoup(response.text)
    links = soup.findAll("div", { "class" : "section_head" })
    c=int(str(links).split(' DONATIONS')[0].split('[<div class="section_head">')[1].strip().replace(",", ""))
    if os.path.isfile('donations/'+page+'.csv'):
        df=pd.read_csv('donations/'+page+'.csv',header=None)
    else:
        for i in reversed(range(c/10+1)):
            idx=i*10
            url='https://www.gofundme.com/mvc.php?route=donate/pagingdonationsb&url='+page+'&idx='+str(idx)
            response=requests.post(url, headers=headers)
            soup = BeautifulSoup(response.text)
            donors = soup.findAll("div", { "class" : "doner" })
            out=open('donations/'+page+'.csv','a')
            for d in reversed(donors):
                line=str(d.findAll("div", { "class" : "dtime" })).split('[<div class="dtime">')[1].split('</div>]')[0].strip()
                tweet= str(d.findAll("div",{ "title": "This donor posted this page to Twitter."}))
                face=str(d.findAll("div",{ "title": "This donor shared this page with Facebook friends."}))
                out.write(line+tweet+face+'\n')
            out.close()
        df=pd.read_csv('donations/'+page+'.csv',header=None)

In [54]:
df=make_df(-1,10000)
df=df[df.url.str.contains('://gofund.me/')]

In [3]:
df['page']=df.url.apply(lambda x: x.split('://gofund.me/')[-1])

In [4]:
dg=df[df.created_at.str.contains('Dec 24')].groupby('page').count().reset_index()
pages=dg.page
len(pages)

61

In [ ]:
for page in pages:
    collect_donations(page)